In [4]:
import pandas as pd
import numpy as np
import ta
from binance import Client
from key import PUB_KEY, SEC_KEY

In [6]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(500) + ' minutes ago UTC')

[[1720510380000,
  '57324.01000000',
  '57324.01000000',
  '57284.46000000',
  '57286.00000000',
  '25.39620000',
  1720510439999,
  '1455365.02780040',
  807,
  '9.14407000',
  '523968.01268810',
  '0'],
 [1720510440000,
  '57285.99000000',
  '57310.58000000',
  '57285.99000000',
  '57292.01000000',
  '13.25166000',
  1720510499999,
  '759279.85179860',
  1164,
  '8.20712000',
  '470240.73105580',
  '0'],
 [1720510500000,
  '57292.01000000',
  '57292.02000000',
  '57225.49000000',
  '57225.50000000',
  '14.97948000',
  1720510559999,
  '857687.12595250',
  741,
  '3.52009000',
  '201593.34863180',
  '0'],
 [1720510560000,
  '57225.50000000',
  '57225.50000000',
  '57160.00000000',
  '57160.00000000',
  '43.51953000',
  1720510619999,
  '2488611.92711450',
  2091,
  '20.64413000',
  '1180478.82186920',
  '0'],
 [1720510620000,
  '57160.01000000',
  '57180.00000000',
  '57119.66000000',
  '57179.99000000',
  '57.29279000',
  1720510679999,
  '3274266.61747540',
  2496,
  '21.26202000',
